<a href="https://colab.research.google.com/github/ElenaGordienko1/NLP/blob/main/NLP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Выпонение лабораторной работы 3

Импорт библиотек ⭐

In [9]:
import numpy as np
import pandas as pd

### Функция линейной регрессии

In [26]:
def linear_regression(X: np.ndarray) -> float:

    weight = np.random.randn()
    bias = np.random.randn()

    return float(X * weight + bias)

### Функция активации: сигмоида





In [27]:
def activation_func(x: float) -> float:

    return 1 / (1 + np.exp(-x))

### Нейрон



In [28]:
def neuron(x: np.ndarray):

    temp_result = linear_regression(x)
    result = activation_func(temp_result)
    return result


### Применение на датасете

In [36]:
df = pd.read_csv("/content/Mall_Customers.csv")

X = df['Annual Income (k$)'].values
Y = df['Spending Score (1-100)'].values

x_sample = X[0]

output = neuron(x_sample)
print(f"Входное значение X: {x_sample}, Выход нейрона: {output}")

Входное значение X: 15, Выход нейрона: 0.7084395994597699
